In [74]:
import pandas as pd
import numpy as np
import scipy.io
import mat73
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
from sklearn.linear_model import LogisticRegression as lg
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score,f1_score
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter,filterwarnings
import os
# ignore all future warnings1
simplefilter(action='ignore', category=FutureWarning)


## EEG data:

In [75]:
 ## data loading
data = mat73.loadmat('x_source.mat')  
id = scipy.io.loadmat('id.mat')
IDs = pd.DataFrame(id["id"])
#flattening and adding id
df2 = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
df2['IDs'] = IDs


In [76]:
df2['IDs']#array of arrays
df2['IDs'][0][0] #this is what i want for 1 guy

'NDARAA075AMK'

In [77]:
#ARd stuff, new array 
df2['IDs'] = [df2['IDs'][i][0] for i in range(2041)]

In [78]:
df2


,0,1,2,3,4,5,6,7,8,9,...,26579,26580,26581,26582,26583,26584,26585,26586,26587,IDs
0,5.078868e-01,5.217506e-01,5.333599e-01,5.438772e-01,5.520926e-01,5.588265e-01,5.634961e-01,5.659676e-01,5.664875e-01,5.649957e-01,...,0.222850,0.217079,0.211663,0.206002,0.200403,0.195159,0.189689,0.184573,0.179243,NDARAA075AMK
1,1.318260e+00,1.346233e+00,1.346461e+00,1.319271e+00,1.277007e+00,1.202095e+00,1.108148e+00,1.024186e+00,9.160927e-01,8.353658e-01,...,0.548031,0.550646,0.548010,0.538900,0.523634,0.507341,0.482457,0.459452,0.427597,NDARAA112DMH
2,3.202934e-01,3.261972e-01,3.312618e-01,3.371723e-01,3.422373e-01,3.481419e-01,3.540378e-01,3.590818e-01,3.649516e-01,3.699674e-01,...,0.741139,0.706200,0.671909,0.627837,0.580535,0.538347,0.488267,0.445411,0.396396,NDARAA117NEJ
3,8.538945e-02,8.781716e-02,9.029479e-02,9.540115e-02,9.803051e-02,1.007110e-01,1.034429e-01,1.062265e-01,1.119492e-01,1.148887e-01,...,0.867153,0.446510,0.271732,0.148466,0.073150,0.032884,0.005997,0.003034,0.002015,NDARAA947ZG5
4,2.198925e-01,2.883633e-01,3.841303e-01,4.960847e-01,5.791005e-01,6.078317e-01,5.715989e-01,4.875744e-01,3.839277e-01,3.020838e-01,...,0.017080,0.016311,0.015609,0.014934,0.014284,0.013658,0.013024,0.012446,0.011891,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2036,5.067916e-01,5.145184e-01,5.211450e-01,5.273791e-01,5.325418e-01,5.371864e-01,5.409880e-01,5.438015e-01,5.459333e-01,5.471581e-01,...,0.431317,0.430619,0.429836,0.428881,0.427792,0.426638,0.425290,0.423890,0.422283,NDARZN277NR6
2037,1.436510e-01,1.486863e-01,1.538399e-01,1.594083e-01,1.648055e-01,1.703211e-01,1.759548e-01,1.817062e-01,1.879041e-01,1.938955e-01,...,0.010595,0.007744,0.006544,0.005896,0.005449,0.005078,0.004725,0.004414,0.004122,NDARZN578YDP
2038,4.393781e-01,4.395668e-01,4.396447e-01,4.396073e-01,4.394592e-01,4.392024e-01,4.388377e-01,4.383663e-01,4.377540e-01,4.370667e-01,...,0.654385,0.649057,0.643871,0.638555,0.633110,0.627543,0.621537,0.615729,0.609810,NDARZN610GTY
2039,1.776357e-15,2.220446e-15,2.664535e-15,3.108624e-15,3.996803e-15,4.884981e-15,5.773160e-15,7.105427e-15,8.881784e-15,1.065814e-14,...,0.041145,0.039611,0.038200,0.036830,0.035499,0.034207,0.032884,0.031669,0.030491,NDARZN677EYE


## Behavioral Data:


In [79]:
behaviour_data = pd.read_csv('scores.csv', low_memory=False)


### We need columns with:
1. IDs

 
2. for Anxiety:  "SCARED_SR" "SCARED_P". 
    


2. for ADHD inattention: Strengths and Weaknesses of ADHD Symptoms and Normal Behavior Scale ("SWAN IN")


3. for ADHD combined: Strengths and Weaknesses of ADHD Symptoms and Normal Behavior Scale ("SWAN HY")


4. for Autism Social Responsiveness Scale-2 ("SRS_Pre" (preschool) or "SRS") 


5. for Depression: Children's Depression Index (average score of: "CDI_SR" "CDI_P") (alternative 1: Mood & Feelings Questionnaire (MFQ); Alternative 2: Affective Reactivity Index—(ARI-S) Self Report)


6. WISC-Full IQ "WISC_FSIQ"


In [80]:
behaviour_data = behaviour_data[['IDs', 'MFQ_P_Total', 'SCARED_P_Total', 'SWAN_HY', "SWAN_IN", 'SRS_Total','ARI_S_Total_Score', 'WISC_FSIQ' ]]

In [81]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns (with column 'IDs').\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

missing_values_table(behaviour_data)

Your selected dataframe has 8 columns (with column 'IDs').
There are 7 columns that have missing values.


,Missing Values,% of Total Values
WISC_FSIQ,588,19.1
MFQ_P_Total,466,15.1
SCARED_P_Total,328,10.7
SRS_Total,218,7.1
SWAN_HY,163,5.3
SWAN_IN,163,5.3
ARI_S_Total_Score,125,4.1


In [82]:

df = pd.merge(df2, behaviour_data, on='IDs', how='inner')
df.shape


(1929, 26596)

In [83]:
print("X:", X.shape, "Y:", Y.shape)
X: (450, 3) Y: (450, 2)


SyntaxError: invalid syntax (1070557066.py, line 2)